## External functions for Fundus of the Eye detecting
Makes the main application clear for user

### Libraries

In [10]:
import cv2
import functools
from IPython.display import Markdown, clear_output, display, HTML
from ipywidgets import widgets, Layout,Label, HBox, VBox, Box
import matplotlib.pyplot as plt
import numpy as np
from os import listdir
from os.path import isfile, join
from skimage.filters import gaussian, threshold_li, sato
from skimage import img_as_float
from skimage.color import rgb2hsv,rgb2gray,hsv2rgb
import skimage.morphology as mp
from sklearn.metrics import roc_curve, auc, confusion_matrix

### Machine Learning Libraries

In [11]:
from imblearn.under_sampling import RandomUnderSampler
import pandas as pd
from pprint import pprint
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_recall_curve, make_scorer, recall_score, accuracy_score, precision_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

### General functions for displaying and loading pictures

In [12]:
def display_picture(image, title=None):
    fig = plt.figure(figsize=(6,6))
    sub = fig.add_subplot(111)
    if title is not None:
        sub.set_title(title)
    plt.axis('off')
    plt.imshow(image) 

def display_results(image_array,titles_array):
    count = len(image_array)
    fig, axs = plt.subplots(nrows=1, ncols=count, figsize=(30,6))
    for i, ax in enumerate(axs.flatten()):
        plt.sca(ax)
        plt.axis('off')
        plt.imshow(image_array[i])
        plt.title(titles_array[i])

    plt.suptitle("Process of finding eye's fundus")
    plt.savefig('img/image-results/test.jpg')
    plt.show()
    
def display_statistics_plot(expect,mask,tpr,fpr,roc_auc):
    fig = plt.figure(figsize=(20,6))
    title = "Classifications metrics"
    fig.suptitle(title, fontsize=16)

    original = fig.add_subplot(131)
    original.set_title('Expert mask')
    plt.axis('off')
    plt.imshow(expect)

    roc_curve = fig.add_subplot(132)
    roc_curve.set_title('Receiver Operating Characteristic')
    roc_curve.plot(fpr, tpr, color='darkorange', label='ROC curve (area = %0.2f)' % roc_auc)
    roc_curve.plot([0, 1], [0, 1], color='navy', linestyle='--')
    xlim=[0.0, 1.0]
    ylim=[0.0, 1.05]
    roc_curve.set_xlim(xlim)
    roc_curve.set_ylim(ylim)
    roc_curve.set_xlabel('False Positive Rate')
    roc_curve.set_ylabel('True Positive Rate')

    actual = fig.add_subplot(133)
    actual.set_title('Obtained mask')
    plt.axis('off')
    plt.imshow(mask)
    
    stats = fig.add_subplot(211)
    stats.axis('tight')
    stats.axis('off')
    labels=list(table.keys())
    text=list(table.values())
    stats.table(cellText=text,colLabels=labels,loc='center')
    
    plt.show()

def display_label_distribution(y_train,y_test):
    classes_number = 2
    fig, ax = plt.subplots()
    training_counts = [None] * classes_number 
    testing_counts = [None] * classes_number
    for i in range(classes_number):
        training_counts[i] = len(y_train[y_train == i])
        testing_counts[i] = len(y_test[y_test == i])

    train_bar = plt.bar(np.arange(classes_number)-0.2, training_counts, align='center', color = 'pink', alpha=0.75, width = 0.41, label='Training')
    test_bar = plt.bar(np.arange(classes_number)+0.2, testing_counts, align='center', color = 'teal', alpha=0.75, width = 0.41, label = 'Testing')
   
    ax.set_xlabel('Labels')
    x_ticks_labels = ['Background','Vessel']
    ax.set_xticks((0,1))
    ax.set_xticklabels(x_ticks_labels,rotation='horizontal', fontsize=14)
    ax.set_ylabel('Count')
    plt.title('Label distribution in the training and test set')
    plt.legend(bbox_to_anchor=(1.05, 1), handles=[train_bar, test_bar], loc=2)
    plt.grid(True)
    plt.show()
    
def load_image(path,file):
    if('.jpg' in file or '.png' or '.tif' in file):
        full_file = path+file
        img = cv2.imread(full_file)
        return img

def apply_pic_on_pic(img1,img2):
    img2=img_as_float(rgb2gray(img2))
    h = img1.shape[0]
    w = img2.shape[1]

    for y in range(h):
        for x in range(w):
            img1[y, x] = 255 if img2[y, x] == 1 else img1[y,x]

    return img1

### Functions for buttons "on-click"
Reacting for using application's buttons

In [19]:
def on_update_clicked(x, img_array, expected_array):
    clear_output()
    display(choose_image_box)
    
    path = 'img/'
    expected_path = 'img/expected/'
    filepath = filename_input.value
    
    img_array.clear()
    img = cv2.imread(path+filepath)
    img_array.append(img)
    
    expected = cv2.imread(expected_path+filepath)
    expected_array.append(expected)
    
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    fig = plt.figure(figsize=(6,6))
    sub = fig.add_subplot(111)
    plt.axis('off')
    plt.imshow(img)

def on_update_many_clicked(x, img_array, expected_array):
    clear_output()
    display(choose_image_box)
    
    path = filepath_input.value
    expected_path = path+'expected/'
    files = [f for f in listdir(path) if isfile(join(path, f))]
    for i,file in enumerate(files):
            img = load_image(path,file)
            img_array.append(img)
                
            expected_img = load_image(expected_path,file)
            expected_array.append(expected_img)
    print(f'Updated {i+1} files!')


def on_find_fundus_clicked(x, images_array, mask):
    if len(images_array)==0:
        display (Markdown('<span style="color: #ff0000">Upload image first!</span>'))
        return
    
    for img in images_array:
        img_orig = img.copy()
        img_orig = cv2.cvtColor(img_orig, cv2.COLOR_BGR2RGB)
        
        initial = initial_processing(img)
        
        edges = edge_detecting(initial,mask)
        detections_mask = vessels_mask(edges)
        final_fundus = fundus_final_processing(detections_mask)
        
        final_fundus = cv2.cvtColor(final_fundus, cv2.COLOR_BGR2RGB)
        result_masks.append(final_fundus)
        
        detection_on_orygin = apply_pic_on_pic(img_orig.copy(),final_fundus)
        
        image_progress=[img_orig,initial,final_fundus,detection_on_orygin]
        titles=['Original picture','Initial processing','Fundus mask','Applied on original']
    
        display_results(image_progress, titles)
   
 
def on_check_stats_clicked(x,result_masks, expected_array):
    accuracy_list,sensitivity_list,precision_list,specificity_list,f_score_list,g_mean_list = calculate_statistics(result_masks, expected_array)
    
    print("Accuracy is inappropriate for imbalanced classification because a high accuracy is achievable by a no skill model that only predicts the majority class.")
    
    print("Metrics that may be useful for imbalanced classification because they focus on one class are: sensitivity-specificity and precision-recall.")
    print("Sensitivity (recall) is  how well the positive class was predicted.")
    print('Specificity is  how well the negative class was predicted.')
    print('Sensitivity and specificity can be combined into a single score that balances both concerns, called the geometric mean.')
    
    print('Precision summarizes the fraction of examples assigned the positive class that belong to the positive class.')
    print("Precision and recall can be combined into a single score that seeks to balance both concerns, called the F-score.")
    
    table = pd.DataFrame({"Accuracy": accuracy_list,"Sensitivity": sensitivity_list,"Specificity": specificity_list, "Geometric mean":g_mean_list,"Precision": precision_list,"Recall":sensitivity_list,"F1":f_score_list})
    display(table)
    
    

def on_extract_image_data_clicked(x,images_array,expected_array):
    block_size = block_size_input.value
    img = images_array[0]
    expected = expected_array[0]

    img = initial_processing(img)
    parts,decisions = split_image(img,expected,block_size)

    data=[]
    for part,decision in zip(parts,decisions):
        data.append(image_features_extract(part,decision))
    data_set = pd.DataFrame(data)
    print('Image based data frame:')
    display(data_set.head())
    data_set.to_csv('data/model_data.csv')
    
def on_train_model_clicked(x):
    bootstrap,n_estimators,criterion,max_depth,max_features,min_samples_leaf,min_samples_split = get_input_model_params()
    train_model(bootstrap,n_estimators,criterion,max_depth,max_features,min_samples_leaf,min_samples_split)
    
def on_find_best_random_clicked(x):
    find_RandomizedSearchCV_best_params()

def on_find_best_grid_clicked(x):
    find_GridSearchCV_best_params()
    
    
def get_input_model_params():
    bootstrap=bootstrap_input.value
    n_estimators=n_estimators_input.value
    criterion=criterion_input.value
    max_depth=max_depth_input.value
    max_features=max_features_input.value
    min_samples_leaf=min_samples_leaf_input.value
    min_samples_split=min_samples_split_input.value
    return bootstrap,n_estimators,criterion,max_depth,max_features,min_samples_leaf,min_samples_split

### Data processing
Functions for calculating the statistics and machine learning

In [14]:
def train_model(bootstrap,n_estimators,criterion,max_depth,max_features,min_samples_leaf,min_samples_split):
    X,y = get_X_y()
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33,stratify=y)
    display_label_distribution(y_train,y_test)
    
    
def get_X_y():
    data_set = pd.read_csv('data/model_data.csv')
    data_set=data_set.drop("Unnamed: 0",axis=1)
    
    X = data_set.iloc[:, :-1].values
    y = data_set.iloc[:, -1].values
    
    rus=RandomUnderSampler()
    X, y = rus.fit_resample(X, y)
    
    return X,y
    

def find_RandomizedSearchCV_best_params():
    # Number of trees in random forest
    n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
    max_features = ['auto', 'sqrt']
    max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
    max_depth.append(None)
    min_samples_split = [2, 5, 10]   # Minimum number of samples required to split a node
    min_samples_leaf = [1, 2, 4]     # Minimum number of samples required at each leaf node
    bootstrap = [True, False]        # Method of selecting samples for training each tree
    
    random_grid = {'n_estimators': n_estimators,
                   'max_features': max_features,
                   'max_depth': max_depth,
                   'min_samples_split': min_samples_split,
                   'min_samples_leaf': min_samples_leaf,
                   'bootstrap': bootstrap }
    
    print("Using the random grid to search for best parameters")
    print("Random Grid options: \n")
    pprint(random_grid)
    
    rf = RandomForestClassifier()
    rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
    # n_iter controls the number of different combinations to try
    # cv is the number of folds to use for cross validation
    
    X, y = get_X_y()
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33,stratify=y)
    
    rf_random.fit(X_train, y_train)
    best_random = rf_random.best_estimator_
    
    print("\nRandom best params:")
    pprint(rf_random.best_params_)
    
    print("\nRandom best estimator:")
    pprint(best_random)

def find_GridSearchCV_best_params():
    param_grid = {
        'bootstrap': [True],
        'max_depth': [80, 90, 100, 110],
        'max_features': [2, 3],
        'min_samples_leaf': [3, 4, 5],
        'min_samples_split': [8, 10, 12],
        'n_estimators': [100, 200, 300, 1000]
    }  #grid parameters based on RandomizedSearchCV best params
    
    
    rf = RandomForestClassifier()
    grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                              cv = 3, n_jobs = -1, verbose = 2)
    
    X, y = get_X_y()
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33,stratify=y)
    
    grid_search.fit(X_train, y_train)
    best_grid = rf_random.best_estimator_
    
    print("\nGrid best params:")
    pprint(rf_random.best_params_)
    
    print("\nGrid best estimator:")
    pprint(best_random)

def calculate_statistics(result_mask, expected_array):
    accuracy_list=[]
    sensitivity_list=[]
    precision_list=[]
    specificity_list=[]
    fpr_list=[]
    f_score_list=[]
    g_mean_list=[]
    
    for mask, expect in zip(result_masks, expected_array): 
        mask_copy, expect_copy = binarize_images(mask.copy(),expect.copy())
        mask_array = mask_copy.flatten()
        expect_array = expect_copy.flatten()
        
        true_negative, false_positive, false_negative, true_positive = confusion_matrix(mask_array, expect_array, labels=[0,1]).ravel()
        fpr, tpr, _ = roc_curve(expect_array, mask_array)
        roc_auc = auc(fpr, tpr)
        
        accuracy =  (true_positive + true_negative) / (true_positive+true_negative+false_positive+false_negative)#trafnosc
        sensitivity =  true_positive / (true_positive + false_negative)#czulosc : tp / (tp + fn)
        precision = true_positive / (true_positive + false_positive) # tp / tp + fp
        specificity = true_negative / (true_negative + false_positive) #swoistosc: tn / (tn + fp)
        f_score = 2 * precision * sensitivity /  ( precision + sensitivity)
        g_mean = np.sqrt(sensitivity*specificity)
        
        accuracy_list.append(accuracy)
        sensitivity_list.append(sensitivity)
        precision_list.append(precision)
        specificity_list.append(specificity)
        f_score_list.append(f_score) 
        g_mean_list.append(g_mean)
        
        display_statistics_plot(expect,mask,tpr,fpr,roc_auc)
        
    return accuracy_list,sensitivity_list,precision_list,specificity_list,f_score_list,g_mean_list

### Manual image processing
Functions for manual processing of the image 

In [15]:
def initial_processing(img):
    lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8,8))
    cl = clahe.apply(l)
    merged = cv2.merge((cl,a,b))
    clahed_image = cv2.cvtColor(merged, cv2.COLOR_LAB2BGR)
    clahed_image[:, :, 0] = 0
    clahed_image[:, :, 2] = 0
    img=img_as_float(rgb2gray(clahed_image))
    img=gaussian(img,sigma=3) 
    img=img**0.2
    img=sato(img) 
    return img

def edge_detecting(img, mask):
    img = (img - np.min(img)) / (np.max(img) - np.min(img))

    img=mp.dilation(img,selem=mp.disk(6))
    img=gaussian(img,sigma=3) 
    img=mp.closing(img, selem=mp.disk(8))
    img=mp.erosion(img,selem=mp.disk(2))

    thresh = threshold_li(img, tolerance=0.0005)
    img = img > thresh

    mask=img_as_float(rgb2gray(mask))
    mask=mp.erosion(mask, selem=mp.disk(5))
    img=img*mask

    return img

def vessels_mask(img):
    detection = np.zeros((img.shape[0], img.shape[1], 3), np.uint8)
    h = img.shape[0]
    w = img.shape[1]

    for y in range(h):
        for x in range(w):
            if cv2.countNonZero(img[y, x]) > 0:
                detection[y, x] = np.array([255, 255, 255], np.uint8)
  
    return detection

def fundus_final_processing(img):
    img = np.array(img)
    img = 255*(cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) > 5).astype('uint8')

    img=mp.erosion(img,selem=mp.disk(1))
    img=mp.closing(img, selem=mp.disk(15))
    
    img = cv2.threshold(img, 100, 255, cv2.THRESH_BINARY)[1]
    
    return img

def binarize_images(mask,expect,value=[1,1,1],thresh=0):
    black=[0,0,0]
    h = expect.shape[0]
    w = expect.shape[1]
    for y in range(h):
        for x in range(w):
            expect_vessel = expect[y,x]
            detect_vessel = mask[y,x]
            if np.amax(expect_vessel) > thresh:
                expect[y,x]=value
                if np.amax(detect_vessel) >thresh:
                    mask[y,x]=value
                else:
                    mask[y,x]=black
            else:
                expect[y,x]=black
                if np.amax(detect_vessel) > thresh:
                    mask[y,x]=value
                else:
                    mask[y,x]=black
    return mask,expect

def split_image(img,expect, block_size):
    windows=[]
    decisions = []
    exp_windows=[]
#     expect,_ = binarize_images(expect,expect,[255,255,255],20)
    for r in range(0,img.shape[0] - block_size, block_size):
        for c in range(0,img.shape[0] - block_size, block_size):
            window = img[r:r+block_size,c:c+block_size]
            exp_window = expect[r:r+block_size,c:c+block_size]
            center = np.amax(exp_window[exp_window.shape[0]//2,exp_window.shape[1]//2])
            if center>50:
                center=1
            else:
                center=0
            exp_windows.append(exp_window)
            decisions.append(center)
            windows.append(window)
                
    return windows,decisions
    
            
def image_features_extract(img, decision):
    data={}
    data['variance'] = np.var(img)
    img=img_as_float(rgb2gray(img))
    moments = cv2.moments(img)
    data={**data,**moments}
    hu_moments = cv2.HuMoments(moments)
    for i,hu in enumerate(hu_moments):
        name = "hu"+str(i)
        data[name]=hu[0]
    data['decision']=decision
    return data
    

### Settings for the buttons and other input options

In [18]:
images_array=[]
image_mask = load_image('img/mask/','01_h_mask.tif')
result_masks = []
expected_array=[]

horizontal_box_layout = Layout(display='flex',
                    flex_flow='row',
                    align_items='stretch',
                    width='100%')

vertical_box_layout = Layout(display='flex',
                    flex_flow='column',
                    align_items='stretch',
                    width='100%')

style = {'description_width': 'initial'}

single_image_label= widgets.Label(value = 'Choose single image to process...')
files = [f for f in listdir("img/") if isfile(join("img/", f))]

filename_input = widgets.Dropdown(
    options=files,
    value='01_h.jpg',
    description='filename',
    disabled=False
)

update_button = widgets.Button(description = "Update image")
update_button.style.button_color = 'lightpink'
update_button.on_click(functools.partial(on_update_clicked, img_array = images_array, expected_array = expected_array))

single_image_box = Box([single_image_label,filename_input,update_button], layout = vertical_box_layout)

many_images_label= widgets.Label(value = '...or process the full directory')
filepath_input = widgets.Text(layout=Layout(width='350px'),description='directory',value='img/')
update_many_button = widgets.Button(description = "Update images")
update_many_button.style.button_color = 'lightpink'
update_many_button.on_click(functools.partial(on_update_many_clicked,img_array = images_array, expected_array = expected_array))


many_images_box = Box([many_images_label,filepath_input,update_many_button], layout = vertical_box_layout)

items = [single_image_box,many_images_box]

choose_image_box = Box(children=items, layout=horizontal_box_layout)

find_fundus_button = widgets.Button(description="Find fundus")
find_fundus_button.style.button_color = 'lightpink'
find_fundus_button.on_click(functools.partial(on_find_fundus_clicked,images_array = images_array,mask = image_mask))

check_stats_button = widgets.Button(description="Check stats!")
check_stats_button.style.button_color = 'lightpink'
check_stats_button.on_click(functools.partial(on_check_stats_clicked,result_masks = result_masks, expected_array = expected_array))

sizes = [4,5,7]

warning_label= widgets.Label(value = 'Choose image to train your model in the upper single image choosing panel')
block_size_label= widgets.Label(value = 'Choose to what size blocks split images')
block_size_input = widgets.Dropdown(
    options=sizes,
    value=4,
    description='blocks size',
    disabled=False
)

extract_data_button = widgets.Button(description="Extract image data")
extract_data_button.style.button_color = 'lightpink'
extract_data_button.on_click(functools.partial(on_extract_image_data_clicked, images_array=images_array, expected_array = expected_array))

prepare_data_box = Box([warning_label,block_size_label,block_size_input,extract_data_button], layout = vertical_box_layout)

warning_random_label= widgets.Label(value = 'Warning! Finding best random parameters may take a few hours')
find_best_random_button = widgets.Button(description="Find best random params")
find_best_random_button.style.button_color = 'lightpink'
find_best_random_button.on_click(functools.partial(on_find_best_random_clicked))

warning_grid_label= widgets.Label(value = 'Faster than random but still slow:')
find_best_grid_button = widgets.Button(description="Find best grid params")
find_best_grid_button.style.button_color = 'lightpink'
find_best_grid_button.on_click(functools.partial(on_find_best_grid_clicked))

find_best_params_box = Box([warning_random_label,find_best_random_button,warning_grid_label,find_best_grid_button], layout = vertical_box_layout)


set_model_params_label = widgets.Label(value='Enter model params or leave default')

bootstrap_input = widgets.Checkbox(
    value=True,
    description='Bootstrap',
    style = style
)

n_estimators_input = widgets.IntSlider(
    value=100,
    min=0,
    max=1000,
    step=100,
    description='Number of trees',
    style = style
)

criterion_input = widgets.Dropdown(
    options=['gini', 'entropy'],
    value='gini',
    description='Criterion',
    style = style
)

max_depth_input = widgets.IntSlider(
    value=8,
    min=10,
    max=100,
    step=10,
    description='Max depth',
    style = style
)

max_features_input = widgets.Dropdown(
    options=['auto', 'sqrt'],
    value='sqrt',
    description='Max features',
)
min_samples_leaf_input = widgets.Dropdown(
    options=['1', '2','4'],
    value='1',
    description='Min samples leaf',
)

min_samples_split_input = widgets.Dropdown(
    options=['2', '5','10'],
    value='2',
    description='Min samples leaf',
)

train_model_button = widgets.Button(description="Train best model!")
train_model_button.style.button_color = 'lightpink'
train_model_button.on_click(functools.partial(on_train_model_clicked))

left_params_box = Box([bootstrap_input,n_estimators_input,criterion_input,max_depth_input],layout = vertical_box_layout)
right_params_box = Box([max_features_input,min_samples_leaf_input,min_samples_split_input],layout = vertical_box_layout)
params_box = Box([left_params_box,right_params_box],layout = horizontal_box_layout)
set_params_box = Box([set_model_params_label,params_box,train_model_button],layout = vertical_box_layout)

